In [20]:
!pip install -q opendatasets
import opendatasets as od

# Download the Kaggle dataset (no API key needed)
od.download("https://www.kaggle.com/datasets/usmanlovescode/india-crop-production-by-state-dataset")

Dataset URL: https://www.kaggle.com/datasets/usmanlovescode/india-crop-production-by-state-dataset


100%|██████████| 1.96M/1.96M [00:00<00:00, 516MB/s]

In [22]:
import os
import pandas as pd

!ls "india-crop-production-by-state-dataset"

# Find the CSV file (if named differently, adjust accordingly)
file_path = "india-crop-production-by-state-dataset/crop_production.csv"
df = pd.read_csv(file_path)

df.head()

crop_production.csv


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [24]:
import os
import pandas as pd

!ls "india-crop-production-by-state-dataset"

# Find the CSV file (if named differently, adjust accordingly)
file_path = "india-crop-production-by-state-dataset/crop_production.csv"
df = pd.read_csv(file_path)

df.head()

crop_production.csv


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [25]:
# Basic cleaning (drop NAs, ensure numeric)
df = df.dropna(subset=['Production', 'Area'])
df = df[(df['Area'] > 0) & (df['Production'] > 0)]

# Create Yield column (tons/hectare)
df['Yield'] = df['Production'] / df['Area']
df.head()


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Yield
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0,1.594896
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0,0.500000
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0,3.147059
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0,3.642045
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0,0.229167


In [27]:
from sklearn.preprocessing import LabelEncoder

le_state = LabelEncoder()
le_crop = LabelEncoder()

df['State_enc'] = le_state.fit_transform(df['State_Name'])
df['Crop_enc'] = le_crop.fit_transform(df['Crop'])

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np # Import numpy for square root

X = df[['State_enc', 'Crop_enc', 'Crop_Year']]
y = df['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds) # Calculate Mean Squared Error
rmse = np.sqrt(mse) # Calculate RMSE by taking the square root of MSE
print(f"✅ RMSE: {rmse:.2f} tons/hectare")

✅ RMSE: 232.43 tons/hectare


In [31]:
import pickle

with open('yield_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('encoders.pkl', 'wb') as f:
    pickle.dump((le_state, le_crop), f)

In [33]:
# Upload Files to Hugging Face Hub
from huggingface_hub import HfApi

api = HfApi()
repo_id = "RameshKolipaka/india-crop-yield-predictor"  # Replace with your HF repo name

# Create repo if not exist (optional)
api.create_repo(repo_id, exist_ok=True)

# Upload files
api.upload_file(
    path_or_fileobj="yield_model.pkl",
    path_in_repo="yield_model.pkl",
    repo_id=repo_id,
    repo_type="model"
)

api.upload_file(
    path_or_fileobj="encoders.pkl",
    path_in_repo="encoders.pkl",
    repo_id=repo_id,
    repo_type="model"
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  yield_model.pkl                       :   2%|1         | 3.21MB /  174MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  encoders.pkl                          : 100%|##########| 2.04kB / 2.04kB            

CommitInfo(commit_url='https://huggingface.co/RameshKolipaka/india-crop-yield-predictor/commit/b938f3a8b735e3fa238066c4031ce08c99265c0e', commit_message='Upload encoders.pkl with huggingface_hub', commit_description='', oid='b938f3a8b735e3fa238066c4031ce08c99265c0e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RameshKolipaka/india-crop-yield-predictor', endpoint='https://huggingface.co', repo_type='model', repo_id='RameshKolipaka/india-crop-yield-predictor'), pr_revision=None, pr_num=None)